## How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from files on the local drive.  All data files can be downloaded from https://www.cse.ust.hk/msbd5003/data/
For example, https://www.cse.ust.hk/msbd5003/data/fruits.txt

In [ ]:
import findspark
findspark.init(spark_home='/mnt/d/spark/spark-3.3.1-bin-hadoop3', python_path='/mnt/d/anacondaForLinux/anaconda/bin/python3', edit_rc=True)

In [ ]:
import findspark
findspark.init(spark_home='/mnt/c/spark/spark-3.3.1-bin-hadoop3', python_path='/mnt/c/anaconda/anaconda/bin/python3')

In [ ]:
# import findspark
# findspark.init()
from pyspark import SparkContext, SparkConf
sc =SparkContext()

your 131072x1 screen size is bogus. expect trouble


23/02/23 23:31:47 WARN Utils: Your hostname, DESKTOP-A resolves to a loopback address: 127.0.1.1; using 172.23.32.217 instead (on interface eth0)
23/02/23 23:31:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/23 23:31:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Read data from local file system:
print(sc.version)

# fruits = sc.textFile('../data/fruits.txt')
# yellowThings = sc.textFile('../data/yellowthings.txt')
fruits = sc.textFile('../data/fruits.txt')
yellowThings = sc.textFile('../data/yellowthings.txt')

3.3.1


In [10]:
print(fruits.collect())
print(yellowThings.collect())

print(fruits.getNumPartitions())

print(fruits.glom().collect())
print(yellowThings.glom().collect())

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'apple', 'apple', 'banana', 'banana', 'orange', 'orange', 'orange', 'grap', 'grap', 'grap', 'grap', 'grap', 'grap', 'grap']
['banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']
2


[['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'apple', 'apple'], ['banana', 'banana', 'orange', 'orange', 'orange', 'grap', 'grap', 'grap', 'grap', 'grap', 'grap', 'grap']]
[['banana', 'bee', 'butter', 'canary melon', 'gold'], ['lemon', 'pineapple', 'sunflower']]


23/02/23 21:02:19 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 794077 ms exceeds timeout 120000 ms
23/02/23 21:02:19 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
# Read data from HDFS :
fruits = sc.textFile('hdfs://url:9000/pathname/fruits.txt')
fruits.collect()

----------

##  RDD operations

In [26]:
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

In [27]:
# fruitsReversed.cache()
# try changing the file and re-execute with and without cache
print(fruitsReversed.collect())
# What happens when you uncomment the first line and run the whole program again with cache()?

#  with doing cache(), no matter the file change or not, data already loaded won't change
# i.e without cache(), collect() would always try to read the original data

['elppa', 'ananab', 'nolem yranac', 'parg', 'nomel', 'egnaro', 'elppaenip', 'yrrebwarts']


In [28]:
# filter

# filter out anything that doesn't meet the requirement
k = 5
shortFruits = fruits.filter(lambda fruit: len(fruit) <= k)
print(shortFruits.collect())

['apple', 'grap', 'lemon']


In [29]:
# flatMap

# split rdd. (map then flatten the rdd)
characters = fruits.flatMap(lambda fruit: list(fruit))
# characters = fruits.flatMap(lambda fruit: list(""))
print(characters.collect())

['a', 'p', 'p', 'l', 'e', 'b', 'a', 'n', 'a', 'n', 'a', 'c', 'a', 'n', 'a', 'r', 'y', ' ', 'm', 'e', 'l', 'o', 'n', 'g', 'r', 'a', 'p', 'l', 'e', 'm', 'o', 'n', 'o', 'r', 'a', 'n', 'g', 'e', 'p', 'i', 'n', 'e', 'a', 'p', 'p', 'l', 'e', 's', 't', 'r', 'a', 'w', 'b', 'e', 'r', 'r', 'y']


In [ ]:
# quiz2 question 4
from operator import add
lines = fruits
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add) \
              .sortBy(lambda x: x[1]) \
              .reduce(lambda x,y: x if x[1] == max(x[1],y[1]) else y)
print(counts)

('orange', 4)


In [ ]:
# quiz2 question 5
from operator import add
lines = fruits
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add) \
              .sortBy(lambda x: x[1]) \
              .filter(lambda x: len(x[0]) >= 5) \
              .reduce(lambda x,y: x if x[1] == max(x[1],y[1]) else y) 
              
print(counts)

('orange', 4)


In [30]:
# union

# concatenate 2 rdd
fruitsAndYellowThings = fruits.union(yellowThings)
print(fruitsAndYellowThings.collect())

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [31]:
# intersection

# intersection between rdd
yellowFruits = fruits.intersection(yellowThings)
print(yellowFruits.collect())

['canary melon', 'pineapple', 'banana', 'lemon']


In [32]:
print(fruits.subtract(yellowThings).collect())

# subtract
# difference between rdd

['grap', 'orange', 'apple', 'strawberry']


In [33]:
# distinct

# remove any duplicate
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
print(distinctFruitsAndYellowThings.collect())

['canary melon', 'grap', 'orange', 'pineapple', 'bee', 'apple', 'strawberry', 'banana', 'sunflower', 'lemon', 'butter', 'gold']


In [6]:
#Cartesian product

# simple cartesian
# return list of tupple ("","")
print(fruits.cartesian(yellowThings).collect())

[('apple', 'banana'), ('apple', 'bee'), ('apple', 'butter'), ('apple', 'canary melon'), ('apple', 'gold'), ('banana', 'banana'), ('banana', 'bee'), ('banana', 'butter'), ('banana', 'canary melon'), ('banana', 'gold'), ('canary melon', 'banana'), ('canary melon', 'bee'), ('canary melon', 'butter'), ('canary melon', 'canary melon'), ('canary melon', 'gold'), ('grap', 'banana'), ('grap', 'bee'), ('grap', 'butter'), ('grap', 'canary melon'), ('grap', 'gold'), ('lemon', 'banana'), ('lemon', 'bee'), ('lemon', 'butter'), ('lemon', 'canary melon'), ('lemon', 'gold'), ('apple', 'lemon'), ('apple', 'pineapple'), ('apple', 'sunflower'), ('banana', 'lemon'), ('banana', 'pineapple'), ('banana', 'sunflower'), ('canary melon', 'lemon'), ('canary melon', 'pineapple'), ('canary melon', 'sunflower'), ('grap', 'lemon'), ('grap', 'pineapple'), ('grap', 'sunflower'), ('lemon', 'lemon'), ('lemon', 'pineapple'), ('lemon', 'sunflower'), ('orange', 'banana'), ('orange', 'bee'), ('orange', 'butter'), ('orange',

In [43]:
# zip 
newyellowThings = yellowThings.repartition(3)
# print(newyellowThings.collect())

print(fruits.zip(yellowThings).collect())
# print(fruits.zip(newyellowThings).collect())  #has differnt number 
# of partition
    # "ZIP" combine two RDD into list of tupples ("","")

# print(fruits.zip(yellowFruits).collect())
    # cannot zip the same RDD

print(fruits.zipWithIndex().collect())
    # combine RDD fruits with index into list of tupples ("", idx)

# Items in the kth partition will get ids k, n+k, 2*n+k, …, 
# where n is the number of partitions. 
print(fruits.zipWithUniqueId().collect())

[('apple', 'banana'), ('banana', 'bee'), ('canary melon', 'butter'), ('grap', 'canary melon'), ('lemon', 'gold'), ('orange', 'lemon'), ('pineapple', 'pineapple'), ('strawberry', 'sunflower')]
[('apple', 0), ('banana', 1), ('canary melon', 2), ('grap', 3), ('lemon', 4), ('orange', 5), ('pineapple', 6), ('strawberry', 7)]
[('apple', 0), ('banana', 2), ('canary melon', 4), ('grap', 6), ('lemon', 8), ('orange', 1), ('pineapple', 3), ('strawberry', 5)]


### RDD actions

In [39]:
# collect
# return a list item of the RDD
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
print(fruitsArray)

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']


In [44]:
# count
# return the size of the RDD
numFruits = fruits.count()
print(numFruits)

8


In [46]:
# take
# retreive the first k elements in Rdd into a list item
first3Fruits = fruits.take(3)
print(first3Fruits)

['apple', 'banana', 'canary melon']


In [3]:
# Tip: Don't use count() when you don't need to return the exact number of rows
# use take() or isEmpty()
print(fruits.isEmpty())

False


In [2]:
print(fruits.map(lambda fruit: len(fruit)).sum())
print(fruits.map(lambda fruit: len(fruit)).reduce(lambda x, y: x+y))

57
57


In [54]:
# the reduce function must be associative; otherwise the result is nondeterministic 
rdd = sc.parallelize([1, 2, 1, 3, 4, 5, 2], 4)
#rdd = sc.parallelize([1, 2, 3], 2)
#print(rdd.glom().collect())
print(rdd.reduce(lambda x, y: 2*x+y))

60


In [48]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
print(letterSet)

{'o', 'r', 'a', 'i', 'p', 'g', 'c', ' ', 'l', 'y', 'e', 'w', 'n', 'b', 'm', 't', 's'}


In [2]:
# treeReduce 
# Data are combined partially on a small set of executors before they are sent to the driver, 
# which dramatically reduces the load the driver has to deal with. 

letterSet = fruits.map(lambda fruit: set(fruit)).treeReduce(lambda x, y: x.union(y))
print(letterSet)

{'o', 'r', 'a', 'i', 'p', 'g', 'c', ' ', 'l', 'y', 'e', 'w', 'n', 'b', 'm', 't', 's'}


In [49]:
letterSet = fruits.flatMap(lambda fruit: list(fruit)).distinct().collect()
print(letterSet)

['p', 'l', 'b', 'c', 'r', 'y', 'g', 'i', 's', 'a', 'e', 'n', ' ', 'm', 'o', 't', 'w']


In [4]:
# fold:
letterSet = fruits.map(lambda fruit: set(fruit)).fold(set(), lambda x, y: x.union(y))
print(letterSet)

{'o', 'r', 'a', 'i', 'p', 'g', 'c', ' ', 'l', 'y', 'e', 'w', 'n', 'b', 'm', 't', 's'}


In [7]:
# reducing an empty rdd is not allowed, but fold is OK.
r = sc.parallelize([])
#r.reduce(lambda x, y: x+y)
r.fold(0, lambda x, y: x+y)

0

In [6]:
# aggregate /  treeAggregate can return a different result type than the type of the RDD

def f(x, y):
    x.add(y)
    return x

letterSet = fruits.flatMap(lambda fruit: list(fruit)).treeAggregate(set(), f, lambda x, y: x.union(y))
print(letterSet)

# It avoids object allocation.
# This is the most efficient way for solving this problem.

{'o', 'r', 'a', 'i', 'p', 'g', 'c', ' ', 'l', 'y', 'e', 'w', 'n', 'b', 'm', 't', 's'}


In [13]:
# foreach is an action, map is a transformation.
fruits.foreach(lambda x: print('I have a', x))

I have a apple
I have a banana
I have a canary melon
I have a grap
I have a lemon
I have a orange
I have a pineapple
I have a strawberry


In [55]:
print(fruits.map(lambda x: 'I have a ' + x).collect())

['I have a apple', 'I have a banana', 'I have a canary melon', 'I have a grap', 'I have a lemon', 'I have a orange', 'I have a pineapple', 'I have a strawberry']


### Closure

In [7]:
counter = 0
rdd = sc.parallelize(range(10), 3)

print(rdd.glom().collect())

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
    print(x, counter)

print(rdd.collect())
rdd.foreach(increment_counter)

print(counter)
print(rdd.sum())

[[0, 1, 2], [3, 4, 5], [6, 7, 8, 9]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
45


03 0 
3
1 14
 72
 35
 12
6 6
7 13
8 21
9 30


In [ ]:
#quiz2 question 5
A = sc.parallelize(range(1, 100))
t = 50
B = A.filter(lambda x: x < t)
t = 10
C = B.filter(lambda x: x > t)
print(C.count())

0


In [ ]:
#quiz2 question 6
A = sc.parallelize(range(1, 100))
t = 50
B = A.filter(lambda x: x < t)
B.cache()#add cache() to get the 
t = 10
C = B.filter(lambda x: x > t)
print(C.count())

39


In [ ]:
#quiz2 question 9
r1 = sc.parallelize([4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5], 3)
r2 = sc.parallelize([1, 3, 4, 5, 6, 7, 3, 5, 4, 9, 3], 3)

print(r1.glom().collect())
print(r2.glom().collect())
def dotProduct(rdd1, rdd2): 
    return (rdd1 
            .zip(rdd2)
            .map(lambda x: x[0]*x[1])
            .reduce(lambda x, y: x + y)
            # .collect()
            )

print(dotProduct(r1, r2))

import numpy as np
print("numpy dotproduct:", np.dot([4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5],[1, 3, 4, 5, 6, 7, 3, 5, 4, 9, 3]))

# r1 .zip(r2) \
#     .map(lambda x: x[0]*x[1]) \
#     .reduce(lambda x, y: x + y)


[[4, 2, 3], [1, 3, 4], [5, 6, 7, 8, 5]]
[[1, 3, 4], [5, 6, 7], [3, 5, 4, 9, 3]]
233
numpy dotproduct: 233
233


In [ ]:
#quiz2 question 10
r1 = sc.parallelize([4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5], 3)
r2 = sc.parallelize([1, 3, 4, 5, 6, 7, 3, 5, 4, 9, 3], 3)

# Induce a different partitioning
r2 = r2.flatMap(lambda x: [] if x == 1 else [x])\
       .flatMap(lambda x: [9, 9] if x == 9 else [x])

#FILL IN YOUR CODE HERE
res = r1.zipWithIndex().map(lambda x:(x[1], x[0]))\
       .join(r2.zipWithIndex().map(lambda x:(x[1], x[0])))\
       .map(lambda x: x[1][0] * x[1][1])\
       .reduce(lambda x, y: x + y)
print(res)

import numpy as np
print("numpy dotproduct:", np.dot([4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5],[3, 4, 5, 6, 7, 3, 5, 4, 9, 9, 3]))

273
numpy dotproduct: 273


In [52]:
rdd = sc.parallelize(range(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x

a = rdd.foreach(g)

print(accum.value)

45


In [66]:
rdd = sc.parallelize(range(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x
    return x * x

a = rdd.map(g)
print(accum.value)
# print(a.reduce(lambda x, y: x+y))
#a.cache()
tmp = a.count()
print(accum.value)
print(rdd.reduce(lambda x, y: x+y))

tmp = a.count()
print(accum.value)
print(rdd.reduce(lambda x, y: x+y))


0
45
45
90
45


In [26]:
n = 100
rdd = sc.parallelize(range(1000000*n) , n)
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x
    return x

a = rdd.map(g)
tmp = a.count()
print(accum.value)

# correct answer: 4999999950000000
# Spark will only update the accumulator from the successful task,and the failed tasks are completely ignored.
# So the accumulator is computed correctly even if some tasks fail

23/02/20 09:08:32 ERROR TaskSchedulerImpl: Lost executor 4 on 143.89.126.35: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/02/20 09:08:33 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() for one-way message.
java.io.InvalidClassException: org.apache.spark.deploy.DeployMessages$ExecutorUpdated; local class incompatible: stream classdesc serialVersionUID = 1654279024112373855, local class serialVersionUID = -1971851081955655249
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:699)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2004)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1851)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2185)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:502)
	at java.io.Obj

4999999950000000


23/02/20 09:14:26 ERROR TaskSchedulerImpl: Lost executor 3 on 143.89.126.34: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/02/20 09:14:27 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() for one-way message.
java.io.InvalidClassException: org.apache.spark.deploy.DeployMessages$ExecutorUpdated; local class incompatible: stream classdesc serialVersionUID = 1654279024112373855, local class serialVersionUID = -1971851081955655249
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:699)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2004)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1851)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2185)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:502)
	at java.io.Obj

In [25]:
n = 100
rdd1 = sc.parallelize(range(1000000*n) , n)
rdd2 = sc.parallelize(range(1000000*n) , n)
rdd = rdd1.zip(rdd2)
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x[0]
    return x

a = rdd.map(g)
b = a.reduceByKey(sum)  # this causes a shuffle
tmp = b.count()
print(accum.value)

# Because shuffle output is stored locally, if a node goes down, that shuffle output is gone. 
# Spark goes back to the stage that generated the shuffle output, looks at which tasks need
# to be rerun, and re-executes them on one of the nodes that is still alive.
# This results in the accumulator being an over-count

# Summary: It's OK to use accumulators in an action (e.g., foreach) but not in a transformation
# Or better: Avoid using them at all.

23/02/20 09:07:44 ERROR TaskSchedulerImpl: Lost executor 2 on 143.89.126.35: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/02/20 09:07:44 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() for one-way message.
java.io.InvalidClassException: org.apache.spark.deploy.DeployMessages$ExecutorUpdated; local class incompatible: stream classdesc serialVersionUID = 1654279024112373855, local class serialVersionUID = -1971851081955655249
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:699)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2004)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1851)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2185)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:502)
	at java.io.Obj

6752499932500000


### Closure and Persistence

In [62]:
# Linear-time selection

data = [34, 67, 21, 56, 47, 89, 12, 44, 74, 43, 26]
A = sc.parallelize(data,2)
k = 4

while True:
    x = A.first()
    A1 = A.filter(lambda z: z < x)
    A2 = A.filter(lambda z: z > x)
    #A1.cache()
    #A2.cache()
    mid = A1.count()
    if mid == k:
        print(x)
        break
    if k < mid:
        A = A1
    else:
        A = A2
        k = k - mid - 1
    #A.cache()

43


In [59]:
sorted(data)

[12, 21, 26, 34, 43, 44, 47, 56, 67, 74, 89]

In [13]:
A = sc.parallelize(range(10))

x = 5
B = A.filter(lambda z: z < x)
#B.cache()
print(B.count())
x = 3
print(B.count())

5
5


In [24]:
A = sc.parallelize(range(10))

x = 5
B = A.filter(lambda z: z < x)
B.cache()  
# In the Scala version of Spark, whether caching B or not will always return [0, 1, 2].
# However, in PySpark, it seems that cache() actually resolves all variables in the transformation (filter in this example)
# at the time the RDD is cached, but not at the time of an action that triggers the transformation.  
# This inconsistency between the Python version and the Scala version of Spark has not been documented.  
# If anyone can find more information, please share with us.
print(B.take(10))
#print(B.collect())

x = 3
print(B.take(10))
#print(B.collect())
# B.collect() doesn't always re-collect data - bad design!
# Always use take() instead of collect()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


In [139]:
# From the official spark examples.
from random import random
from operator import add

partitions = 100
n = 100000 * partitions

def f(_: int) -> float:
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))


[Stage 94:=========================>                            (48 + 48) / 100]



Pi is roughly 3.144800


In [160]:
from random import random

a = sc.parallelize(range(0,10),2)
print(a.map(lambda _: random()).glom().collect())

# Why are random numbers in all partitions the same?

[[0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095], [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095]]


In [163]:
random()

0.763774618976614

### mapPartitions and mapPartitionsWithIndex

In [3]:
# mapPartitions

def f(pa):
    l = []
    for x in pa:
        l.append(x[::-1])
    return l
        
fruitsReversed = fruits.mapPartitions(f)
print(fruitsReversed.collect())

['elppa', 'ananab', 'nolem yranac', 'parg', 'nomel', 'egnaro', 'elppaenip', 'yrrebwarts']


In [50]:
# It's more efficient to use yield

def f(pa):
    for x in pa:
        yield x[::-1]

fruitsReversed = fruits.mapPartitions(f)
print(fruitsReversed.collect())

#  It provides a facility to do heavy initializations (for example Database connection) once for each partition
# instead of doing it on every element in the RDD.

['elppa', 'ananab', 'nolem yranac', 'parg', 'nomel', 'egnaro', 'elppaenip', 'yrrebwarts']


In [6]:
# Can also do some transformation on the partition level

def f(pa):
    return sorted(pa, reverse = True)

print(fruits.glom().collect())
print(fruits.mapPartitions(f).collect())

[['apple', 'banana', 'canary melon', 'grap', 'lemon'], ['orange', 'pineapple', 'strawberry']]
['lemon', 'grap', 'canary melon', 'banana', 'apple', 'strawberry', 'pineapple', 'orange']


In [7]:
# mapPartitionsWithIndex

def f(i, pa):
    for x in pa:
        yield str(i) + x

fruitsReversed = fruits.mapPartitionsWithIndex(f)
print(fruitsReversed.collect())

# These two functions will be useful for advanced algorithm design (will see later)

['0apple', '0banana', '0canary melon', '0grap', '0lemon', '1orange', '1pineapple', '1strawberry']


In [82]:
# Correct version for computing Pi
from random import random, seed
from time import time

partitions = 1
n = 100000 * partitions

s = time()

def f(index, it):
    seed(index + s)
    for i in it:
        x = random() * 2 - 1
        y = random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 <= 1 else 0

count = sc.parallelize(range(1, n + 1), partitions).mapPartitionsWithIndex(f).sum()

print("Pi is roughly", 4.0 * count / n)

Pi is roughly 3.1394


In [109]:
# Check that random numbers are different
from random import random, seed
from time import time

s = time()

def f(index, it):
    seed(index + s)
    for i in it:
        yield random()

print(sc.parallelize(range(10), 2).mapPartitionsWithIndex(f).glom().collect())

[[0.8143627602226674, 0.05568968411039621, 0.06350599524003342, 0.6437126318558297, 0.582409005280173], [0.12353092242461161, 0.3906074815540721, 0.8548366905291449, 0.3000379392845135, 0.7906655356347023]]


### Key-Value Pairs

In [15]:
# groupByKey
groupFruitsByLength = fruits.map(lambda fruit: (len(fruit), fruit)).groupByKey()
print(groupFruitsByLength.take(10))
for x in groupFruitsByLength.take(1)[0][1]:
    print(x)

[(6, <pyspark.resultiterable.ResultIterable object at 0x7f8bc619c2e8>), (12, <pyspark.resultiterable.ResultIterable object at 0x7f8bc619c0b8>), (4, <pyspark.resultiterable.ResultIterable object at 0x7f8bc619cc50>), (10, <pyspark.resultiterable.ResultIterable object at 0x7f8bc619ca58>), (5, <pyspark.resultiterable.ResultIterable object at 0x7f8bc61a89b0>), (9, <pyspark.resultiterable.ResultIterable object at 0x7f8bc61a85f8>)]
banana
orange


In [17]:
# count the number of fruits by length: mimics MapReduce

print(fruits.map(lambda fruit: (len(fruit), 1)).groupByKey().mapValues(sum).collect())

[(6, 2), (12, 1), (4, 1), (10, 1), (5, 2), (9, 1)]


In [18]:
# reduceByKey: this more efficient
# reduceByKey will compute local sums for each key in each partition and combine those local sums 
# into larger sums after shuffling.

numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)
print(numFruitsByLength.take(10))

# aggregateByKey, foldByKey also available.
# but there is no treeAggregateByKey

[(6, 2), (12, 1), (4, 1), (10, 1), (5, 2), (9, 1)]


In [12]:
from operator import add

lines = sc.textFile('../data/course.txt')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)
print(counts.take(20))

Py4JJavaError: An error occurred while calling o95.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/mnt/c/workspace/OneDrive - HKUST Connect/hkust/data/course.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.api.java.JavaRDDLike.parti

In [35]:
print(counts.sortBy(lambda x: x[1], False).take(20))

# print(counts.sortBy(lambda x: x[1], True).take(20))
# acending order

[('data', 4), ('of', 3), ('and', 3), ('Course', 2), ('in', 2), ('the', 2), ('Information', 1), ('systems,', 1), ('cloud', 1), ('parallel', 1), ('as', 1), ('mining', 1), ('massive', 1), ('amount', 1), ('even', 1), ('servers', 1), ('centers.', 1), ('both', 1), ('hands-on', 1), ('this', 1)]


In [11]:
print(counts.sortByKey().take(20))

NameError: name 'counts' is not defined

In [37]:
print(counts.collectAsMap())

{'Course': 2, 'Information': 1, 'systems,': 1, 'cloud': 1, 'parallel': 1, 'as': 1, 'in': 2, 'mining': 1, 'massive': 1, 'amount': 1, 'of': 3, 'even': 1, 'servers': 1, 'centers.': 1, 'both': 1, 'hands-on': 1, 'this': 1, 'new': 1, 'Lecture': 1, 'videos': 1, 'Description': 1, 'Big': 1, 'data': 4, 'including': 1, 'computing': 1, 'and': 3, 'processing': 1, 'frameworks,': 1, 'emerge': 1, 'enabling': 1, 'technologies': 1, 'managing': 1, 'the': 2, 'across': 1, 'hundreds': 1, 'or': 1, 'thousands': 1, 'commodity': 1, 'This': 1, 'course': 1, 'exposes': 1, 'students': 1, 'to': 1, 'theory': 1, 'experience': 1, 'technology.': 1}


In [41]:
counts.lookup('data')
# This scans the whole RDD, unless there is a partitioner (to be discussed later)

[4]

### Join vs. Broadcast Variables

In [27]:
# Join simple example

products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
#trans = sc.parallelize([(1, 134, "OK"), (3, 34, "OK"), (5, 162, "Error"), (1, 135, "OK"), (2, 53, "OK"), (1, 45, "OK")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(products.join(trans).take(20))

[(1, ('Apple', (134, 'OK'))), (1, ('Apple', (135, 'OK'))), (1, ('Apple', (45, 'OK'))), (2, ('Orange', (53, 'OK'))), (3, ('TV', (34, 'OK'))), (5, ('Computer', (162, 'Error')))]


In [27]:
products = {1: "Apple", 2: "Orange", 3: "TV", 4: "PC", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

broadcasted_products = sc.broadcast(products)

results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
#  results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print(results.take(20))


[(1, 'Apple', (134, 'OK')), (3, 'TV', (34, 'OK')), (5, 'Computer', (162, 'Error')), (1, 'Apple', (135, 'OK')), (2, 'Orange', (53, 'OK')), (1, 'Apple', (45, 'OK'))]


In [33]:
# Compare with cogroup

products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (4, "PC"), (5, "Computer")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

for x,y in products.cogroup(trans).collect():
    print(x, tuple(map(list, y)))

1 (['Apple'], [(134, 'OK'), (135, 'OK'), (45, 'OK')])
2 (['Orange'], [(53, 'OK')])
3 (['TV'], [(34, 'OK')])
4 (['PC'], [])
5 (['Computer'], [(162, 'Error')])


### K-means clustering

In [26]:
import numpy as np

def parseVector(line):
    return np.array([float(x) for x in line.split()])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_data.txt
lines = sc.textFile('../data/kmeans_data.txt', 5)  

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_bigdata.txt
# lines = sc.textFile('../data/kmeans_bigdata.txt', 5)  
# lines is an RDD of strings
K = 3
convergeDist = 0.01  
# terminate algorithm when the total distance from old center to new centers is less than this value

data = lines.map(parseVector).cache() # data is an RDD of arrays

kCenters = data.takeSample(False, K, 1)  # intial centers as a list of arrays
tempDist = 1.0  # total distance from old centers to new centers

while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kCenters), (p, 1)))
    # for each point in data, find its closest center
    # closest is an RDD of tuples (index of closest center, (point, 1))
        
    pointStats = closest.reduceByKey(lambda p1, p2: (p1[0] + p2[0], p1[1] + p2[1]))
    # pointStats is an RDD of tuples (index of center,
    # (array of sums of coordinates, total number of points assigned))
    
    newCenters = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()
    # compute the new centers
    
    tempDist = sum(np.sum((kCenters[i] - p) ** 2) for (i, p) in newCenters)
    # compute the total disctance from old centers to new centers
    
    for (i, p) in newCenters:
        kCenters[i] = p
        
print("Final centers: ", kCenters)


Final centers:  [array([0.1       , 0.33333333, 0.23333333]), array([9.05, 3.05, 4.65]), array([9.2, 2.2, 9.2])]


### PageRank

In [41]:
import re
from operator import add

def computeContribs(urls, rank):
    # Calculates URL contributions to the rank of other URLs.
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    # Parses a urls pair string into urls pair."""
    parts = urls.split(' ')
    return parts[0], parts[1]

# Loads in input file. It should be in format of:
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/*
lines = sc.textFile("../data/pagerank_data.txt", 2)
# lines = sc.textFile("../data/dblp.in", 5)

numOfIterations = 2
# Loads all URLs from input file and initialize their neighbors. 
links = lines.map(lambda urls: parseNeighbors(urls)) \
             .groupByKey()

# Loads all URLs with other URL(s) link to from input file 
# and initialize ranks of them to one.
ranks = links.mapValues(lambda neighbors: 1.0)

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(numOfIterations):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks) \
                    .flatMap(lambda url_urls_rank:
                             computeContribs(url_urls_rank[1][0],
                                             url_urls_rank[1][1]))
    # After the join, each element in the RDD is of the form
    # (url, (list of neighbor urls, rank))
    
    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(\
        lambda rank: rank * 0.85 + 0.15)
    # ranks = contribs.reduceByKey(add).map(lambda t: (t[0], t[1] * 0.85 + 0.15))


print("final:",ranks.top(5, lambda x: x[1]))

final: [('2', 17889.982504323973), ('3', 17162.090081483562), ('4', 11433.74304797224), ('5', 9211.558991129874), ('1', 8771.999190287586)]


In [5]:
ranks = links.mapValues(lambda neighbors: 1.0)
tmp = links.join(ranks)
tmp.collect()

[('1', (<pyspark.resultiterable.ResultIterable at 0x7f588c057610>, 1.0)),
 ('4', (<pyspark.resultiterable.ResultIterable at 0x7f588c07b4f0>, 1.0)),
 ('2', (<pyspark.resultiterable.ResultIterable at 0x7f587c4eb490>, 1.0)),
 ('3', (<pyspark.resultiterable.ResultIterable at 0x7f587c4eb5b0>, 1.0))]